# 1. Import Statements:

In [ ]:
pip install contractions

In [ ]:
pip install labMTsimple

In [ ]:
pip install marisa-trie

In [ ]:
pip install word2number

In [ ]:
pip install nameparser

In [ ]:
import os
import csv
import math
import numpy as np
import codecs
import re
import matplotlib.pyplot as plt
import contractions
import itertools
import spacy
from spacy.lang.en import English
from scipy.sparse import lil_matrix
from labMTsimple.speedy import LabMT
from word2number import w2n
from nameparser import HumanName
import nltk
from nltk.corpus import names

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Creating an instance of the LabMT class
LabMT = LabMT()

# 1. Function to get the title of the book:

In [ ]:
def get_title(book):

    # Getting the Title of the book
    pattern1 = "Title:\s(.*)"
    match = re.search(pattern1, book.read())
    title = match.group(1)

    return title.strip()

# 2. Function to remove the first and last part of the book (Unnecessary):

In [ ]:
def clean_end_first(book, filename):

    # Removing the first part of the book
    pattern2 = "\*\*\*\s?START.*?\s?\*\*\*"
    book_clean = re.sub("^(?s:.*?)" + pattern2, " ", book.read(), count = 1)

    # Removing the last part of the book
    pattern3 = "\*\*\*\s?END.*?\s?\*\*\*"
    book_clean = re.sub(pattern3 + "(?s:.*)", " ", book_clean, count = 1)

    # Removing [Illustrations: ...] from the books
    pattern4 = "\[Illustration:(.*)\]"
    book_clean = re.sub(pattern4, "", book_clean)

    # Removing the text "The End" and others from the book
    pattern5 = "(?i)the end|end of the project gutenberg ebook of(.*)|end of project gutenberg's(.*)"
    book_clean = re.sub(pattern5, "", book_clean)

    # On Manual inspection, these books had the list of all books in the Wizard of Oz Series
    # This Regex will be used to remove that
    if filename == "8. Tik-Tok of Oz.txt" or filename == "9. The Scarecrow of Oz.txt" or filename == "10. Rinkitink in Oz.txt" or filename == "12. The Tin Woodman of Oz.txt" or filename == "14. Glinda of Oz.txt":
        pattern6 = "(The Wonderful Oz Books|THE FAMOUS OZ BOOKS)"
        book_clean = re.sub(pattern6 + "(?s:.*)", " ", book_clean)

    return book_clean

# 3. Function to Find all the Chapters Titles in the book:

In [ ]:
def chapter_titles(book, filename):
    # Finding all the chapter names in the book
    dictTOC = {}
    chapter_title = []

    if filename == "3. Ozma of Oz.txt" or filename == "4. Dorothy and the Wizard in Oz.txt" or filename == "5. The Road to Oz.txt" or filename == "13. The Magic of Oz.txt":
        # Pattern seen in Book 3, 4, 5, 13
        pattern7 = re.findall("(\d+)(\.)(\s+)(.*)", book)

        for i in range(len(pattern7)):
            title = pattern7[i][0] + pattern7[i][1] + pattern7[i][2] + pattern7[i][3]
            if len(pattern7[i][0]) <= 2:
                dictTOC[pattern7[i][0]] = title.strip()
    else:
        pattern8 = re.findall("(Chapter)(.*)", book)

        for i in range(len(pattern8)):
            title = pattern8[i][0] + pattern8[i][1]
            key_val = w2n.word_to_num(pattern8[i][1])
            dictTOC[key_val] = title.strip()

    for key in dictTOC:
        number = int(key) - 1
        chapter_title.insert(number, dictTOC[key])

    return chapter_title

# 4. Getting the text for each chapter in the book:

In [ ]:
def matching_chapter_titles(chapter_title, book_clean, filename, val):

    matches = re.finditer(re.escape(chapter_title), book_clean)
    count = 0
    temp = list(matches)
    length = len(temp)

    for i in range(length):
        if length == 2:
            match_index = temp[1].span()[val]
        else:
            match_index = temp[0].span()[val]

    return match_index

In [ ]:
def chapter_chunck(chapter_titles, book_clean, filename):

    chapter_chuncks = []

    for i in range(len(chapter_titles)):

        # Read till the next chapter title
        if i != (len(chapter_titles) - 1):

            # Getting the chapter title to start and end with
            chapter_title = chapter_titles[i]
            next_chapter_title = chapter_titles[i + 1]

            # Finding the positions in the text
            # This will get the chapter chunck

            # Start position
            end_of_start_chapter_title_index = matching_chapter_titles(chapter_title, book_clean, filename, 1)

            # End Position
            start_of_next_chapter_title_index = matching_chapter_titles(next_chapter_title, book_clean, filename, 0)

            # Getting the chapter chuncks and putting it into a list
            chapter_chuncks.append(book_clean[end_of_start_chapter_title_index:start_of_next_chapter_title_index])

        # Read till the end
        else:
            # This is done for the last chapter, where we will read till the end of the book

            chapter_title = chapter_titles[i]
            end_of_start_chapter_title_index = matching_chapter_titles(chapter_title, book_clean, filename, 1)
            chapter_chuncks.append(book_clean[end_of_start_chapter_title_index:])

    return chapter_chuncks

# 5. Function to remove paragraph spacing and weird line breaks:

In [ ]:
def remove_spacing_line_breaks(chapter_chuncks):

    # Each paragraph is separated using \r\n\r\n
    # Each new line is created using \r\n
    # Both of these will now be replaced using \s
    # Dealing with contractions as well

    chapter_chuncks_clean = []

    for chapter in chapter_chuncks:
        chapter = chapter.strip()
        chapter = chapter.replace("\r\n", " ")
        chapter = chapter.replace("\r\n\r\n", " ")
        chapter = contractions.fix(chapter)
        chapter_chuncks_clean.append(chapter)

    return chapter_chuncks_clean

# 6. Function to perform Sentence Tokenisation:

In [ ]:
def sent_tokenise(chapter_chuncks_clean):
    # Using SpaCy to perform sentence Tokenization

    sent_tokenizer = spacy.load("en_core_web_sm")
    all_sents_in_each_chapter = []

    for chapter in chapter_chuncks_clean:
        sent_chapter = []
        sentences = sent_tokenizer(chapter)

        for sent in sentences.sents:
            sent_chapter.append(sent)

        all_sents_in_each_chapter.append(sent_chapter)

    return all_sents_in_each_chapter

# 7. Function to perform Word Tokenisation:

In [ ]:
def word_tokenise(chapter_chuncks_clean):
    # Using SpaCy to perform word Tokenization
    # Loading the English Tokenizer

    word_tokenizer = English()
    all_words_in_each_chapter = []

    for chapter in chapter_chuncks_clean:
        words_chapter = []
        words = word_tokenizer(chapter.strip())
        for word in words:
            if not word.is_punct and len(word) != 0 and word.text.strip() != "":
                words_chapter.append(word.text.lower())
        all_words_in_each_chapter.append(words_chapter)

    return all_words_in_each_chapter

# 12. Function to create a dictionary with the frequency of each word:

In [ ]:
# Function to get the count of each word in the text

def word_freq(words_in_text):
    count_dict = {}

    for word in words_in_text:
        if word in count_dict:
            count_dict[word] += 1
        else:
            count_dict[word] = 1

    return count_dict

# 13. Stopper Function:

In [ ]:
# This function was taken directly from the Documentation written by the authors
# Referencing the author's work as there were issues with accessing this function
# Reference: Mitchell L. Kiley D. Danforth C.M. Reagan, A.J. and P.S. Dodds. The emotional arcs of stories are dominated by six basic shapes, 2016.

def stopper(tmpVec,score_list,word_list,stopVal=1.0,ignore=[],center=5.0):
    """Take a frequency vector, and 0 out the stop words.

    Will always remove the nig* words.

    Return the 0'ed vector."""

    ignoreWords = ["nigga","nigger","niggaz","niggas"];
    for word in ignore:
        ignoreWords.append(word)
    newVec = tmpVec
    for i in range(len(score_list)):
        if abs(score_list[i]-center) < stopVal:
            newVec[i] = 0
        if word_list[i] in ignoreWords:
            newVec[i] = 0

    return newVec

# 14. Function to Generate the Emotion Arc:

In [ ]:
def generate_emotional_arc(all_word_list, LabMT, window_size, num_of_chuncks, title, length):

    step_size = int(math.floor((len(all_word_list) - window_size) / (num_of_chuncks - 1)))
    # Creating a frequency matrix for each chunck of text (100)

    # Along the row will be the number of chuncks of text
    # Along the columns will be the words in the dictionary (10222 in total)
    # We will create a sparse frequency matrix

    freq_matrix = lil_matrix((num_of_chuncks, len(LabMT.scorelist)))

    for i in range(num_of_chuncks):

        if i != num_of_chuncks - 1:

            # Get the word frequency dictionary for each chunck of text
            word_freq_dict = word_freq(all_word_list[(i * step_size):(window_size + (i * step_size))])

            # Converting the frequency matrix into a sparse representation using wordVecify
            freq_matrix[i, :] = LabMT.wordVecify(word_freq_dict)

        else:
            word_freq_dict = word_freq(all_word_list[(i * step_size):])
            freq_matrix[i, :] = LabMT.wordVecify(word_freq_dict)

    # Generating the emotion Arcs

    emotion_score = list(np.zeros((num_of_chuncks)))
    X = []

    for i in range(num_of_chuncks):
        X.append(i)

    for i in range(num_of_chuncks):

        # Convert each row of the sparse matrix representation into a list
        text_vec = freq_matrix[i, :].toarray().squeeze()

        stoppedVec = stopper(text_vec,LabMT.scorelist,LabMT.wordlist)

        # Getting the scores based on the frequency of the words appearance in the chunck
        emotion_score[i] = np.dot(LabMT.scorelist,stoppedVec)/sum(stoppedVec)

    # Saving the emotion scores
    with open("/content/drive/MyDrive/The Wizard of Oz/Emotional Arc - Trial 1/" + title + ".csv", "a", newline = "") as csvf:
      csv_w = csv.writer(csvf)

      csv_w.writerow(["window_number", "emotion_score"])

      for element1, element2 in zip(X, emotion_score):
          csv_w.writerow([element1, element2])

    # Plotting the Emotion Arc
    plt.plot(X, emotion_score)
    plt.title(title + " - " + str(length) + " words")
    plt.xlabel("Percentage of Book")
    plt.ylabel("Emotional Score based on Valence")

    # Displaying the Emotion Arc
    # plt.savefig("/content/drive/MyDrive/The Wizard of Oz/Emotional Arc - Trial 1/" + title + ".pdf")
    plt.show()
    print()
    print()

# 15. Plotting the Predicted and Gold Standard Emotion Arcs:

In [ ]:
def evaluate(title, file):

  path = "/content/drive/MyDrive/The Wizard of Oz/Emotional Arc - Trial 1/" + title + "/" + title
  X1 = []
  X2 = []
  emotional_score1 = []
  emotional_score2 = []

  # Reading the Generated Score
  with open(path + ".csv", "r") as csvf:
    csvr = csv.reader(csvf)
    for row in csvr:
      if len(row) >= 2:
        X1.append(row[0])
        emotional_score1.append(row[1])
  X1 = X1[1:]
  emotional_score1 = emotional_score1[1:]
  X1 = [int(x) for x in X1]
  emotional_score1 = [float(x) for x in emotional_score1]
  X1 = [x for x, score in zip(X1, emotional_score1) if score != 0]
  emotional_score1 = [score for score in emotional_score1 if score != 0]

  # Reading the Evaluation Score
  with open(path + " - Gold.csv", "r") as csvf:
    csvr = csv.reader(csvf)
    for row in csvr:
      if len(row) >= 2:
        X2.append(row[0])
        emotional_score2.append(row[1])
  X2 = X2[1:]
  emotional_score2 = emotional_score2[1:]
  X2 = [int(x) for x in X2]
  emotional_score2 = [float(x) for x in emotional_score2]
  X2 = [x for x, score in zip(X2, emotional_score2) if score != 0]
  emotional_score2 = [score for score in emotional_score2 if score != 0]

  # Calculating and saving the mean squared error
  error = []
  for i in range(len(emotional_score2)):
    temp = (emotional_score2[i] - emotional_score1[i]) ** 2
    error.append(temp)
  mean_sqaured_error = sum(error) / len(emotional_score2)

  # Saving the Mean Squared Error Values
  text = "MSE of " + title + ": " + str(mean_sqaured_error)
  with open("/content/drive/MyDrive/The Wizard of Oz/Emotional Arc - Trial 1/Mean Sqaured Error Values.txt", "a") as f:
    f.write(text)
    f.write("\n")

  # Generating the plots
  figure, axes = plt.subplots(nrows = 1, ncols = 2, figsize = (12, 5))

  axes[0].plot(X1, emotional_score1)
  axes[0].set_xlabel("Percentage of Book")
  axes[0].set_ylabel("Emotional Score based on Valence")
  axes[0].set_title("Generated Emotion Arc")

  axes[1].plot(X2, emotional_score2)
  axes[1].set_xlabel("Percentage of Book")
  axes[1].set_ylabel("Emotional Score based on Valence")
  axes[1].set_title("Gold Standard Emotion Arc")

  figure.suptitle(title, fontsize = 32)
  plt.tight_layout()
  plt.savefig("/content/drive/MyDrive/The Wizard of Oz/Emotional Arc - Trial 1/" + title + "/" + title + " - Comparison.pdf")
  plt.show()
  print()
  print()

# 16. Main Function:

In [ ]:
path = "/content/drive/MyDrive/The Wizard of Oz"
counter = 0

# Getting all the books in the Folder
for root, _, files in os.walk(path):
    for file in files:
        if file != ".DS_Store":

          b = os.path.join(root, file)
          # Opening the books using codecs
          book = codecs.open(b, "r", encoding = "utf-8")

          # Getting the Title of the book
          title = get_title(book)

          # Removing the uneccesary information related to Project Gutenberg
          # at the beginning and end of the book
          book = codecs.open(b, "r", encoding = "utf-8")
          book_clean = clean_end_first(book, file)

          # Getting all the chapter titles from the book in order
          chapter_title = chapter_titles(book_clean, file)

          # Getting the text associated with each chapter in order
          chapter_chuncks = chapter_chunck(chapter_title, book_clean, file)

          # Removing paragraph spacing and line breaks
          chapter_chuncks_clean = remove_spacing_line_breaks(chapter_chuncks)

          # Sentence Tokenisation for each chapter in book
          all_sents_in_each_chapter = sent_tokenise(chapter_chuncks_clean)

          # Getting all the words in the whole text
          all_sents_list = list(itertools.chain(*all_sents_in_each_chapter))

          # Word Tokenisation for each chapter in book
          all_words_in_each_chapter = word_tokenise(chapter_chuncks_clean)

          # Getting all the words in the whole text
          all_word_list = list(itertools.chain(*all_words_in_each_chapter))

          # The number of words in the book
          length_of_book = len(all_word_list)

          # Emotion Arc Generation
          emotion_score = generate_emotional_arc(all_word_list, LabMT, 10000, 100, title, length_of_book)

          # Evaluating the Emotion Arcs against the Gold Standard Emotion Arcs
          evaluate(title, file)